In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Dec 10 01:28:15 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       On  | 00000000:00:1E.0 Off |                    0 |
| N/A   36C    P0              32W /  70W |   8109MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install accelerate>0.20.2
!pip install tensorboardX
!pip install transformers[torch]
!pip install -q git+https://github.com/huggingface/peft.git@main
!pip install -i https://test.pypi.org/simple/ bitsandbytes
!pip install accelerate

In [2]:
!pip install boto3
!pip install sagemaker
!pip install botocore
!pip install fastapi
!pip install s3fs
!pip install -q transformers datasets librosa evaluate jiwer gradio accelerate 
!pip install -q git+https://github.com/huggingface/peft.git@main

  Using cached fastapi-0.95.2-py3-none-any.whl.metadata (24 kB)
  Using cached pydantic-1.10.13-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (149 kB)
Using cached fastapi-0.95.2-py3-none-any.whl (56 kB)
Using cached pydantic-1.10.13-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.5.2
    Uninstalling pydantic-2.5.2:
      Successfully uninstalled pydantic-2.5.2
  Attempting uninstall: fastapi
    Found existing installation: fastapi 0.104.1
    Uninstalling fastapi-0.104.1:
      Successfully uninstalled fastapi-0.104.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 4.8.0 requires pydantic>=2.0, but you have pydantic 1.10.13 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages th

In [ ]:
!python -m bitsandbytes
!cd bitsandbytes
!CUDA_VERSION=121 make cuda12x


In [3]:
!pip install tensorboardX

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
from datasets import load_dataset
import s3fs

# Define your S3 bucket path
s3_bucket_path = 's3://project-whisper-data/dataset/'

# Use s3fs to list files in the bucket
s3 = s3fs.S3FileSystem()
files_in_bucket = s3.ls(s3_bucket_path)

# Print the list of files
print("Files in the S3 bucket:")
print(files_in_bucket)

#s3.get(s3_bucket_path, './whisper_data_tr/', recursive=True, maxdepth=None)


Files in the S3 bucket:
['project-whisper-data/dataset/dataset_dict.json', 'project-whisper-data/dataset/test', 'project-whisper-data/dataset/train']


In [6]:
from datasets import load_dataset, DatasetDict
common_voice = DatasetDict.load_from_disk("whisper_data_tr/dataset")
s3_bucket_path = 's3://project-whisper-data/dataset/'
#common_voice = DatasetDict.load_from_disk(s3_bucket_path)
#cc = common_voice['test'].train_test_split(test_size=0.01)

In [7]:
#type(cc['train'])

In [8]:
model = "openai/whisper-small"

In [9]:
from transformers import WhisperFeatureExtractor
feature_extractor = WhisperFeatureExtractor.from_pretrained(model)

from transformers import WhisperTokenizer
tokenizer = WhisperTokenizer.from_pretrained(model, language="Turkish", task="transcribe")

from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained(model, language="Turkish", task="transcribe")

In [10]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [11]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [12]:
import evaluate

metric = evaluate.load("wer")

In [13]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [15]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("whisper-small-tr-cp/checkpoint-4000")#,load_in_8bit=True,device_map="auto")

In [16]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

from peft import prepare_model_for_kbit_training,prepare_model_for_int8_training

model = prepare_model_for_kbit_training(model)

def make_inputs_require_grad(module, input, output):
    output.requires_grad_(True)

model.model.encoder.conv1.register_forward_hook(make_inputs_require_grad)

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

#config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

In [22]:
from transformers import Seq2SeqTrainingArguments
checkpoint_bucket = 's3://project-whisper-output/'
s3_bucket_path = 's3://project-whisper-data/dataset/'

training_args = Seq2SeqTrainingArguments(
    output_dir=  "./whisper-small-tr-cp2",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-4,
    warmup_steps=500,
    max_steps=10_000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    do_eval = True,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=2500,
    eval_steps=2500,
    logging_steps=100,
    report_to=["tensorboard"],
    load_best_model_at_end=False,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
    #remove_unused_columns=False,  # required as the PeftModel forward doesn't have the signature of the wrapped model's forward
    #label_names = ['labels']
)

In [23]:
from transformers import TrainerCallback
checkpoint_bucket = 's3://project-whisper-output'
class MyCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, **kwargs):
        path = f"checkpoints/cp-{state.global_step}"
        torch.save(kwargs["model"], path)

In [24]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(args=training_args,
                            model=model,
                            train_dataset=common_voice["train"],
                            eval_dataset=common_voice["test"],
                            data_collator=data_collator,
                            compute_metrics=compute_metrics,
                            tokenizer=processor.feature_extractor,
                            )

In [25]:
processor.save_pretrained(training_args.output_dir)

In [26]:
trainer.train()

Step,Training Loss,Validation Loss,Wer
2500,0.155500,0.755009,95.608469
5000,0.073100,0.664064,65.952533


KeyboardInterrupt: 

In [28]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0 TR",  # a 'pretty' name for the training dataset
    "dataset_args": "config: tr, split: test",
    "language": "tr",
    "model_name": "Whisper Small Hi TR",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [29]:
trainer.push_to_hub(**kwargs)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1702172307.ip-172-16-90-82.eu-north-1.compute.internal:   0%|          | 0.00/13.5k [00:00…

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

'https://huggingface.co/Kiwipirate/whisper-small-tr-cp2/tree/main/'

In [34]:
from datasets import load_dataset
import s3fs

# Define your S3 bucket path
s3_bucket_path = 's3://project-whisper-output'

# Use s3fs to list files in the bucket
s3 = s3fs.S3FileSystem()
files_in_bucket = s3.ls(s3_bucket_path)

# Print the list of files
print("Files in the S3 bucket:")
print(files_in_bucket)

s3.put('./whisper-small-tr-cp2/', s3_bucket_path, recursive=True, maxdepth=None)


Files in the S3 bucket:
[]
